In [5]:
import requests
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
import os
from dateutil import tz

# Rough working in notebook

In [46]:
AEROAPI_BASE_URL = "https://aeroapi.flightaware.com/aeroapi/flights/"
AEROAPI_KEY = os.environ.get("FA_API_KEY")
AEROAPI_KEY = '' # NB you will need to get this because os.environ.get isn't working in notebook!!!! 
AEROAPI = requests.Session()
AEROAPI.headers.update({"x-apikey": AEROAPI_KEY})

In [64]:
flight_number = 'UAL4'
params = {'start':datetime.date(2022,11,20)}

In [53]:
full_url = AEROAPI_BASE_URL + flight_number
response = AEROAPI.get(full_url, params = params).json()


In [54]:
print(response)

{'flights': [{'ident': 'UAL4', 'ident_icao': 'UAL4', 'ident_iata': 'UA4', 'fa_flight_id': 'UAL4-1668838520-fa-0007', 'operator': 'UAL', 'operator_icao': 'UAL', 'operator_iata': 'UA', 'flight_number': '4', 'registration': 'N17963', 'atc_ident': None, 'inbound_fa_flight_id': 'UAL923-1668778602-fa-0000', 'codeshares': ['ACA5931', 'AUA7813', 'BEL9080', 'DLH7621', 'SWR3038'], 'codeshares_iata': ['AC5931', 'OS7813', 'SN9080', 'LH7621', 'LX3038'], 'blocked': False, 'diverted': False, 'cancelled': False, 'position_only': False, 'origin': {'code': 'EGLL', 'code_icao': 'EGLL', 'code_iata': 'LHR', 'code_lid': None, 'timezone': 'Europe/London', 'name': 'London Heathrow', 'city': 'London', 'airport_info_url': '/airports/EGLL'}, 'destination': {'code': 'KIAH', 'code_icao': 'KIAH', 'code_iata': 'IAH', 'code_lid': 'IAH', 'timezone': 'America/Chicago', 'name': "Houston Bush Int'ctl", 'city': 'Houston', 'airport_info_url': '/airports/KIAH'}, 'departure_delay': 0, 'arrival_delay': 0, 'filed_ete': 36600, 

# Turning it into functions

In [314]:
def get_raw_flight_details(flight_number, date):
    '''Enter a flight number e.g UAL4, and a date of travel and return a json blob
    containing flight details. 
    The data will be raw and in need of post processing using other functions. 
    '''
    AEROAPI_BASE_URL = "https://aeroapi.flightaware.com/aeroapi/flights/"
    AEROAPI_KEY = os.environ.get("FA_API_KEY")
    AEROAPI_KEY = 'Iduztx3iwprJuplEkG3h9rA9oNzpGeWt' # NB you will need to get this because os.environ.get isn't working in notebook!!!! 
    AEROAPI = requests.Session()
    AEROAPI.headers.update({"x-apikey": AEROAPI_KEY})
    
    start = date
    end = date + datetime.timedelta(days = 1)
    
    full_url = AEROAPI_BASE_URL + flight_number
    params = {'start':start,
              'end':end
             }
    
    response = AEROAPI.get(full_url, params = params)    
    
    response_json = response.json()
    
    if response.ok:   
        if len(response_json['flights']) == 1:
            return response_json['flights']
        if len(response_json['flights']) == 0:
            print('Cannot find a flight with that Flight Number / Date combo. Please try again...')
        if len(response_json['flights']) > 1:
            print('More than 1 flight returned... Please filter your search further...')
    else:
        print('Bad API response')
        return 'Bad API response'


In [315]:
get_raw_flight_details(flight_number, datetime.date(2022,11,22))

[{'ident': 'UAL4',
  'ident_icao': 'UAL4',
  'ident_iata': 'UA4',
  'fa_flight_id': 'UAL4-1668924917-fa-0012',
  'operator': 'UAL',
  'operator_icao': 'UAL',
  'operator_iata': 'UA',
  'flight_number': '4',
  'registration': 'N45956',
  'atc_ident': None,
  'inbound_fa_flight_id': 'UAL923-1668869345-fa-0000',
  'codeshares': ['ACA5931', 'AUA7813', 'BEL9080', 'DLH7621', 'SWR3038'],
  'codeshares_iata': ['AC5931', 'OS7813', 'SN9080', 'LH7621', 'LX3038'],
  'blocked': False,
  'diverted': False,
  'cancelled': False,
  'position_only': False,
  'origin': {'code': 'EGLL',
   'code_icao': 'EGLL',
   'code_iata': 'LHR',
   'code_lid': None,
   'timezone': 'Europe/London',
   'name': 'London Heathrow',
   'city': 'London',
   'airport_info_url': '/airports/EGLL'},
  'destination': {'code': 'KIAH',
   'code_icao': 'KIAH',
   'code_iata': 'IAH',
   'code_lid': 'IAH',
   'timezone': 'America/Chicago',
   'name': "Houston Bush Int'ctl",
   'city': 'Houston',
   'airport_info_url': '/airports/KIAH

In [ ]:
get_raw_flight_details('NOFLIGHT', datetime.date(2022,11,20))

Bad API response


In [305]:
def localize_time(time, timezone):
    to_zone = tz.gettz(timezone)
    return time.astimezone(to_zone)

In [306]:
def add_local_times(raw_details):
    '''
    Take raw flight details json blob and create columns with the the scheduled departure
    and arrival times, localized to the dep/arr locations to match the training data
    '''
    columns = ['scheduled_out',
           'scheduled_in',
           'operator_iata',
           'flight_number',
           'origin',
           'destination',
           'filed_ete',
           'route_distance']
    
    raw_details_df = pd.DataFrame(raw_details)[columns]
    raw_details_df['scheduled_out'] = pd.to_datetime(raw_details_df['scheduled_out'])
    raw_details_df['scheduled_in'] = pd.to_datetime(raw_details_df['scheduled_in'])

    raw_details_df['origin_code'] = raw_details_df.origin[0]['code_iata']
    raw_details_df['origin_tz'] = raw_details_df.origin[0]['timezone']

    raw_details_df['dest_code'] = raw_details_df.destination[0]['code_iata']
    raw_details_df['dest_tz'] = raw_details_df.destination[0]['timezone']
    
    raw_details_df['scheduled_out_local'] = raw_details_df.apply(lambda raw_details: localize_time(raw_details.scheduled_out, 
                                                       raw_details.origin_tz), 
                                                       axis = 1)
    raw_details_df['scheduled_in_local'] = raw_details_df.apply(lambda raw_details: localize_time(raw_details.scheduled_in, 
                                                      raw_details.dest_tz), 
                                                      axis = 1)
    return raw_details_df


In [307]:
def extract_info_from_datetime_col(df):
    '''
    Given a dataframe of raw flight details, now in local time,
    extract year, month, day, weekday, and format local time of dep/arr.
    
    '''
    df['Year'] = df['scheduled_out_local'].dt.year
    df['Quarter'] = df['scheduled_out_local'].dt.quarter
    df['Month'] = df['scheduled_out_local'].dt.month
    df['DayofMonth'] = df['scheduled_out_local'].dt.day
    df['DayOfWeek'] = df['scheduled_out_local'].dt.dayofweek + 1
    df['CRSDepTime'] = df['scheduled_out_local'].dt.strftime('%H%M')
    df['CRSArrTime'] = df['scheduled_in_local'].dt.strftime('%H%M')
    df['FlightDate'] = df['scheduled_out_local'].dt.date
    return df


In [308]:
def clean_df(df):
    '''
    Get the output df we need to feed into model for preprocessing.
    Rename the columns where needed. 
    '''
    columns = ['FlightDate',
               'Year',
               'Quarter',
               'Month',
               'DayofMonth',
               'DayOfWeek',
               'operator_iata', #turn into 'Marketing_Airline_Network'
               'origin_code', # change to Origin
               'dest_code', # change to Dest
               'CRSDepTime',
               'CRSArrTime',
               'route_distance' # change to 'Distance'
              ]
    df_out = df[columns]
    df_out = df_out.rename(columns = {'operator_iata':'Marketing_Airline_Network',
                             'origin_code':'Origin',
                             'dest_code':'Dest',
                             'route_distance':'Distance'})
    
    return df_out
               

In [309]:
def get_processed_flight_details(flight_number, date):
    raw_details = get_raw_flight_details(flight_number, date)
    if raw_details == 'Bad API response':
        print('There has been an issue connecting to the Flightaware API')
        return
    raw_details_localised = add_local_times(raw_details)
    expanded_date_details = extract_info_from_datetime_col(raw_details_localised)
    return clean_df(expanded_date_details)

In [310]:
get_processed_flight_details(flight_number, datetime.date(2022,11,22))

,FlightDate,Year,Quarter,Month,DayofMonth,DayOfWeek,Marketing_Airline_Network,Origin,Dest,CRSDepTime,CRSArrTime,Distance
0,2022-11-22,2022,4,11,22,2,UA,LHR,IAH,1345,1815,4829


In [2]:
from delai.api.flightaware import get_processed_flight_details

df = get_processed_flight_details()
df

,FlightDate,Year,Quarter,Month,DayofMonth,DayOfWeek,Marketing_Airline_Network,Origin,Dest,CRSDepTime,CRSArrTime,Distance
0,2022-12-06,2022,4,12,6,2,YX,EWR,PHL,817,917,80


In [3]:
df.shape

(1, 12)